In [ ]:
import scrapy

class AldibotSpider(scrapy.Spider):
    name = 'aldibot'
    allowed_domains = ['aldi-nord.de']
    start_urls = [
        'https://www.aldi-nord.de/sortiment.html'
    ]

    def parse(self, response):
        # Find all category links on the main Sortiment page
        category_links = response.css('a.category-tile__link::attr(href)').getall()
        for link in category_links:
            yield response.follow(link, callback=self.parse_category)

    def parse_category(self, response):
        # Find all product links on the category page
        product_links = response.css('a.product-tile__image-link::attr(href)').getall()
        for link in product_links:
            yield response.follow(link, callback=self.parse_product)

        # Pagination: follow next page if available
        next_page = response.css('a.pagination__link--next::attr(href)').get()
        if next_page:
            yield response.follow(next_page, callback=self.parse_category)

    def parse_product(self, response):
        product_name = response.css('h1.product-detail-header__title::text').get()
        image_url = response.css('img.product-detail-slider__image::attr(src)').get()
        price_whole = response.css('span.price__main-value::text').get()
        price_decimal = response.css('span.price__decimal-value::text').get()
        price = f"{price_whole}{price_decimal}" if price_whole and price_decimal else None
        product_url = response.url

        yield {
            'product_name': product_name.strip() if product_name else None,
            'image_url': response.urljoin(image_url) if image_url else None,
            'product_url': product_url,
            'price': price,
        }